# EDA on NYC Taxi Tip Data

In [0]:
# Load data
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NYC Taxi Tip EDA").getOrCreate()
df = spark.read.csv("/FileStore/tables/NYC_Taxi_Trip_Record_Clean.csv", header=True, inferSchema=True)
df.cache()

Out[35]: DataFrame[lpep_pickup_datetime: string, lpep_dropoff_datetime: string, day_category: string, pickup_time: string, RatecodeID: string, PULocationID: int, PUBorough: string, PUZone: string, DOLocationID: int, DOBorough: string, DOZone: string, passenger_count: string, trip_distance: double, distance_bins: string, fare_amount: double, fare_amount_bins: string, tip_amount: double, tip_amount_bins: string, payment_type: string, trip_type: string, trip_duration: double, duration_bins: string, speed: double, speed_bins: string]

In [0]:
# 2. 
df.printSchema()

root
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- day_category: string (nullable = true)
 |-- pickup_time: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- PUBorough: string (nullable = true)
 |-- PUZone: string (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- DOBorough: string (nullable = true)
 |-- DOZone: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- distance_bins: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- fare_amount_bins: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tip_amount_bins: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)
 |-- trip_duration: double (nullable = true)
 |-- duration_bins: string (nullable = true)
 |-- speed: double (n

In [0]:

df.show(5)

+--------------------+---------------------+------------+-----------+-------------+------------+---------+-------------------+------------+---------+--------------------+---------------+-------------+-------------+-----------+----------------+----------+---------------+------------+-----------+-------------+-------------+-----+----------+
|lpep_pickup_datetime|lpep_dropoff_datetime|day_category|pickup_time|   RatecodeID|PULocationID|PUBorough|             PUZone|DOLocationID|DOBorough|              DOZone|passenger_count|trip_distance|distance_bins|fare_amount|fare_amount_bins|tip_amount|tip_amount_bins|payment_type|  trip_type|trip_duration|duration_bins|speed|speed_bins|
+--------------------+---------------------+------------+-----------+-------------+------------+---------+-------------------+------------+---------+--------------------+---------------+-------------+-------------+-----------+----------------+----------+---------------+------------+-----------+-------------+---------

In [0]:

df.count()

Out[4]: 60698

In [0]:
# 5. Get summary statistics (count, mean, stddev, min, max) of numeric columns?

df.describe().show()

+-------+--------------------+---------------------+------------+-----------+-------------+------------------+-------------+--------------------+-----------------+-------------+--------------------+------------------+------------------+-------------+------------------+----------------+------------------+---------------+------------+-----------+------------------+-------------+-----------------+----------+
|summary|lpep_pickup_datetime|lpep_dropoff_datetime|day_category|pickup_time|   RatecodeID|      PULocationID|    PUBorough|              PUZone|     DOLocationID|    DOBorough|              DOZone|   passenger_count|     trip_distance|distance_bins|       fare_amount|fare_amount_bins|        tip_amount|tip_amount_bins|payment_type|  trip_type|     trip_duration|duration_bins|            speed|speed_bins|
+-------+--------------------+---------------------+------------+-----------+-------------+------------------+-------------+--------------------+-----------------+-------------+-----

In [0]:
# 6. Calculate average tip amount grouped by passenger count
# TODO: Write the code to answer the above question

from pyspark.sql.functions import avg
df.groupBy("passenger_count").agg(avg("tip_amount").alias("avg_tip_amount")).show()



+---------------+------------------+
|passenger_count|    avg_tip_amount|
+---------------+------------------+
|              3| 2.309636576787808|
|              1|2.2478845308828332|
|             >3| 2.434799190010125|
|              2|2.2726701916721734|
+---------------+------------------+



In [0]:
# 7. Calculate total tip amount by payment type
# TODO: Write the code to answer the above question

from pyspark.sql.functions import sum
df.groupBy("payment_type").agg(sum("tip_amount").alias("total_tip_amount")).show()



+------------+-----------------+
|payment_type| total_tip_amount|
+------------+-----------------+
|        Cash|              0.0|
|     Dispute|              0.0|
|   No charge|              0.0|
| Credit card|137161.1000000002|
|     Unknown|              0.0|
+------------+-----------------+



In [0]:
# 8. Display records where the tip amount is greater than 5
# TODO: Write the code to answer the above question

df.filter(df.tip_amount > 5).show()


+--------------------+---------------------+------------+-----------+---------------+------------+---------+--------------------+------------+---------+--------------------+---------------+-------------+-------------+-----------+----------------+----------+---------------+------------+-----------+-------------+-------------+-----+----------+
|lpep_pickup_datetime|lpep_dropoff_datetime|day_category|pickup_time|     RatecodeID|PULocationID|PUBorough|              PUZone|DOLocationID|DOBorough|              DOZone|passenger_count|trip_distance|distance_bins|fare_amount|fare_amount_bins|tip_amount|tip_amount_bins|payment_type|  trip_type|trip_duration|duration_bins|speed|speed_bins|
+--------------------+---------------------+------------+-----------+---------------+------------+---------+--------------------+------------+---------+--------------------+---------------+-------------+-------------+-----------+----------------+----------+---------------+------------+-----------+-------------+

In [0]:
# 9. Identify outliers where tip amount is greater than 50?
# TODO: Write the code to answer the above question

df.filter(df.tip_amount > 50).show()



+--------------------+---------------------+------------+-----------+----------+------------+---------+------+------------+---------+------+---------------+-------------+-------------+-----------+----------------+----------+---------------+------------+---------+-------------+-------------+-----+----------+
|lpep_pickup_datetime|lpep_dropoff_datetime|day_category|pickup_time|RatecodeID|PULocationID|PUBorough|PUZone|DOLocationID|DOBorough|DOZone|passenger_count|trip_distance|distance_bins|fare_amount|fare_amount_bins|tip_amount|tip_amount_bins|payment_type|trip_type|trip_duration|duration_bins|speed|speed_bins|
+--------------------+---------------------+------------+-----------+----------+------------+---------+------+------------+---------+------+---------------+-------------+-------------+-----------+----------------+----------+---------------+------------+---------+-------------+-------------+-----+----------+
+--------------------+---------------------+------------+-----------+----

In [0]:
# 10. How to calculate the correlation between trip distance and tip amount?
# TODO: Write the code to answer the above question

correlation = df.stat.corr("trip_distance", "tip_amount")
print("Correlation between trip distance and tip amount:", correlation)



Correlation between trip distance and tip amount: 0.4165306370446274


In [0]:
# 11. Get average tip amount by day of the week
# TODO: Write the code to answer the above question

from pyspark.sql.functions import date_format, avg

df.groupBy("day_category").agg(avg("tip_amount").alias("avg_tip_amount")).show()

df.withColumn("day_of_week", date_format("lpep_pickup_datetime", "E")) \
  .groupBy("day_of_week") \
  .agg(avg("tip_amount").alias("avg_tip_amount")) \
  .show()



+------------+------------------+
|day_category|    avg_tip_amount|
+------------+------------------+
|    Weekdays|2.2449286887939603|
|     Weekend| 2.304612369627317|
+------------+------------------+

+-----------+------------------+
|day_of_week|    avg_tip_amount|
+-----------+------------------+
|       null|2.2597301393785663|
+-----------+------------------+



In [0]:
# 12. Get average tip amount by hour of the day
# TODO: Write the code to answer the above question

from pyspark.sql.functions import hour, avg

df.withColumn("pickup_hour", hour("lpep_pickup_datetime")) \
  .groupBy("pickup_hour") \
  .agg(avg("tip_amount").alias("avg_tip_amount")) \
  .orderBy("pickup_hour") \
  .show()


+-----------+------------------+
|pickup_hour|    avg_tip_amount|
+-----------+------------------+
|       null|2.2597301393785663|
+-----------+------------------+



In [0]:
# 13. Calculate tip amount per mile and describe its statistics
# TODO: Write the code to answer the above question

from pyspark.sql.functions import col

df_with_tip_per_mile = df.withColumn("tip_per_mile", col("tip_amount") / col("trip_distance"))

df_filtered = df_with_tip_per_mile.filter((col("trip_distance") > 0) & (col("tip_amount").isNotNull()))

df_filtered.select("tip_per_mile").describe().show()



+-------+------------------+
|summary|      tip_per_mile|
+-------+------------------+
|  count|             60698|
|   mean|0.9462572701376127|
| stddev|0.9968029260390485|
|    min|               0.0|
|    max|17.857142857142858|
+-------+------------------+



In [0]:
# 14. Get records with invalid fare or tip amounts. Then remove these invalid records from dataframe (make it clean)
# TODO: Write the code to answer the above question

from pyspark.sql.functions import col

invalid_records = df.filter(
    (col("fare_amount") <= 0) | 
    (col("tip_amount") < 0) | 
    col("fare_amount").isNull() | 
    col("tip_amount").isNull()
)
invalid_records.show()

clean_df = df.filter(
    (col("fare_amount") > 0) & 
    (col("tip_amount") >= 0) &
    col("fare_amount").isNotNull() &
    col("tip_amount").isNotNull()
)


+--------------------+---------------------+------------+-----------+----------+------------+---------+------+------------+---------+------+---------------+-------------+-------------+-----------+----------------+----------+---------------+------------+---------+-------------+-------------+-----+----------+
|lpep_pickup_datetime|lpep_dropoff_datetime|day_category|pickup_time|RatecodeID|PULocationID|PUBorough|PUZone|DOLocationID|DOBorough|DOZone|passenger_count|trip_distance|distance_bins|fare_amount|fare_amount_bins|tip_amount|tip_amount_bins|payment_type|trip_type|trip_duration|duration_bins|speed|speed_bins|
+--------------------+---------------------+------------+-----------+----------+------------+---------+------+------------+---------+------+---------------+-------------+-------------+-----------+----------------+----------+---------------+------------+---------+-------------+-------------+-----+----------+
+--------------------+---------------------+------------+-----------+----

In [0]:
#15. Calculate the average tip by trip type
from pyspark.sql.functions import avg

avg_tip_by_trip_type_pd = df.groupBy("trip_type").agg(avg("tip_amount").alias("avg_tip_amount")).toPandas()
avg_tip_by_trip_type_pd



,trip_type,avg_tip_amount
0,Street-hail,2.252948
1,Dispatch,3.004161


In [0]:
# 16. Get average tip amount per hour. Show graph as visualization
from pyspark.sql.functions import col, hour, avg, to_timestamp

df = df.withColumn("pickup_hour", hour(to_timestamp("lpep_pickup_datetime", "MM-dd-yyyy H.mm"))) \
       .groupBy("pickup_hour") \
       .agg(avg(col("tip_amount").cast("double")).alias("avg_tip"))

df.display()

pickup_hour,avg_tip
12,2.0196501220504475
22,2.1918211920529798
1,2.1970234113712372
13,1.9839053254437882
6,2.5077810650887575
16,2.330386121992166
3,1.7932258064516127
20,2.078037475345166
5,2.2277083333333336
19,2.1376011560693624


Databricks visualization. Run in Databricks to view.

In [0]:
# 17. Get average tip by passenger count and display as bar chart
from pyspark.sql.functions import col, avg

df.groupBy("passenger_count") \
  .agg(avg(col("tip_amount").cast("double")).alias("avg_tip")) \
  .display()

passenger_count,avg_tip
3,2.309636576787808
1,2.2478845308828332
>3,2.434799190010125
2,2.2726701916721734


Databricks visualization. Run in Databricks to view.

In [0]:
# 18. Get fare vs tip values and display scatter plot style visualization
from pyspark.sql.functions import col

fare_tip_df = df.select(col("fare_amount").cast("double"), col("tip_amount").cast("double"))
display(fare_tip_df)

fare_amount,tip_amount
14.9,4.03
10.7,2.64
6.5,1.7
6.0,0.0
17.7,0.0
19.1,4.85
14.2,0.0
7.2,1.0
24.7,3.0
26.8,0.0


Databricks visualization. Run in Databricks to view.